## **PREPROCESSING RVO DATASET**

## TABLE OF CONTENTS:
- [IMPORTS](#imports)
- [LOAD DATASET](#load_data)
- [CLEAN PROJECT DATA](#project_data)
    - [START DATES](#start_dates)
    - [DROP EMPTY ROWS](#drop_rows)
    - [FUNDING AMOUNT](#funding)
- [TEXT PREPROCESSING FOR BERT](#text_prep)
- [TRANSLATION](#translation)
- [EXPORT DATA](#export)

## IMPORTS <a class="anchor" id="imports"></a>

In [1]:
import pandas as pd
from IPython.display import display
import re
from langdetect import detect
from tqdm.notebook import tqdm
from deep_translator import GoogleTranslator

## LOAD DATASET <a class="anchor" id="load_data"></a>

In [ ]:
# filters used in dataset 
# IMPORT DATA RVO (https://projecten.topsectorenergie.nl/projecten) FILTER SELECTIE OP BASIS MAIL VAN PBL OP 3-Jun-2024**\
# filter op: “zoeken op TKI’s en Thema’s”: aangeklikt:
# 5 restwarmte industrie
# Directe effecten aan de schoorsteen (scope 1)
# Transitieagenda kunststoffen
# 3 CO2 afvang en (her)gebruik (CCU)
# 4 Procesverbetering/energiebesparing
# 6 Circulair
# Duurzaam grondstoffengebruik
# Transitieagenda circulaire bouweconomie
# Transitieagenda maakindustrie
# 7 Biobased/biomassa
# Energie en Industrie
# TKI BBE
# Transitieagenda Consumptiegoederen

In [ ]:
# Load dataset
df_rvo = pd.read_excel("../data/rvo_export.xlsx")

In [3]:
# View information of dataset
display(df_rvo.head(2))
print(df_rvo.info())

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,"€ 1.549.579,00",Kingspan Insulation B.V.,Kingspan Insulation B.V.,12 electrical-driven drying ovens,"Aanleiding\nCurrently, the Kingspan's Koolther...",https://projecten.topsectorenergie.nl/projecte...
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,"€ 489.962,00",Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",2D Circulair Biobased Bouwsysteem,Aanleiding\nOm circulaire bouwmethodes goedkop...,https://projecten.topsectorenergie.nl/projecte...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Subsidie categorie     785 non-null    object
 1   TKI/Thema              785 non-null    object
 2   Innovatiethema('s)     785 non-null    object
 3   Subsidieregeling       785 non-null    object
 4   Projectnummer          785 non-null    object
 5   Looptijd               785 non-null    object
 6   Subsidiebedrag         785 non-null    object
 7   Penvoerder             778 non-null    object
 8   Projectconsortium      756 non-null    object
 9   Projecttitel           785 non-null    object
 10  Publieke samenvatting  768 non-null    object
 11  Website link           785 non-null    object
dtypes: object(12)
memory usage: 73.7+ KB
None


## CLEAN PROJECT DATA <a class="anchor" id="project_data"></a>

### START DATES <a class="anchor" id="start_dates"></a>

In [4]:
# Extract start date from column 'Looptijd' and add as new columns 'start_year' and 'start_date'
df_rvo['start_year'] = df_rvo['Looptijd'].str.extract(r'(\d{4})')
df_rvo['start_date'] = df_rvo['start_year'] + '-01-01'
df_rvo

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link,start_year,start_date
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,"€ 1.549.579,00",Kingspan Insulation B.V.,Kingspan Insulation B.V.,12 electrical-driven drying ovens,"Aanleiding\nCurrently, the Kingspan's Koolther...",https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,"€ 489.962,00",Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",2D Circulair Biobased Bouwsysteem,Aanleiding\nOm circulaire bouwmethodes goedkop...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01
2,Versnelde klimaatinvesteringen industrie (VEKI),5. Restwarmte industrie,Lokale distributie van warmte,VEKI-19-01-01-Klimaatinvest. Indus.,VEKI120016,01-07-2020 - 31-03-2021,"€ 253.118,00",4Pet Recycling B.V.,4PET Recycling B.V.,4PET op stoom,Aanleiding\n4PET Recycling verwerkt kunststof ...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01
3,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI220005,08-10-2020 - 31-12-2022,"€ 3.000.000,00",Coöperatie Koninklijke Cosun UA,Coöperatie Koninklijke Cosun U.A.*,8 Trapsverdamping,Aanleiding\nCosun Beet Company (CBC) beschikt ...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01
4,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI221016,01-01-2022 - 31-01-2024,"€ 200.173,00",Delicia B.V.,Delicia B.V.,Add-on warmtepomp op bestaande koelinstallatie,Aanleiding\nIn de toekomst wil Delicia geen aa...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,Demonstratie Energie Innovatie (DEI),TKI BBE,Biotechnologische conversietechnologie,DEI-16-02-01-Tender 2,DEI2160277,01-11-2016 - 31-05-2019,"€ 5.831.166,00",Protix Biosystems B.V.,Protix B.V.,"Zero Emission, Energy Efficient Protein Produc...",Aanleiding\nThe world population will reach 9....,https://projecten.topsectorenergie.nl/projecte...,2016,2016-01-01
781,EZ Energie Innovatie,3. CO2 afvang en (her)gebruik (CCU),CO2 afvang en (her)gebruik (CCU),TSE-19-17-02-CCUS Topsector Energiestudies,TESN219002,08-04-2019 - 31-08-2020,"€ 496.256,00",Tata Steel Nederland Technology B.V.,"Tata Steel Nederland Technology B.V., TNO",Zero Emissions Ironmaking Process - Phase 1 of...,Aanleiding\nTATA Steel IJmuiden (TSIJ) is resp...,https://projecten.topsectorenergie.nl/projecte...,2019,2019-01-01
782,EZ Energie Innovatie,Duurzaam grondstoffengebruik,Circulaire kunststoffen,TSE Industrie studies 2022,TESN-22-01-02973326,07-04-2022 - 06-10-2023,"€ 273.000,00",UPPACT B.V.,NaN,Ziekenhuisafval Unwasted,Aanleiding\nNederlandse ziekenhuizen producere...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01
783,Demonstratie Energie Innovatie (DEI),Transitieagenda Maakindustrie,Transitieagenda Maakindustrie,DEI-19-03-07-Circulaire Economie URG,DEI2720036,13-08-2020 - 31-10-2021,"€ 675.527,00",Elemetal B.V.,Elemetal B.V.,Zinkloging uit bodemassen,Aanleiding\nZink staat in de top 5 van meest s...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01


### DROP EMPTY ROWS <a class="anchor" id="drop_rows"></a>

In [5]:
# Drop rows where 'Publieke samenvatting' is empty
df_rvo = df_rvo.dropna(subset=['Publieke samenvatting'])

# Drop all rows where 'Projectconsortium' is empty
df_rvo = df_rvo.dropna(subset=['Projectconsortium'])

In [6]:
# Check
print(df_rvo.info())

<class 'pandas.core.frame.DataFrame'>
Index: 739 entries, 0 to 784
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Subsidie categorie     739 non-null    object
 1   TKI/Thema              739 non-null    object
 2   Innovatiethema('s)     739 non-null    object
 3   Subsidieregeling       739 non-null    object
 4   Projectnummer          739 non-null    object
 5   Looptijd               739 non-null    object
 6   Subsidiebedrag         739 non-null    object
 7   Penvoerder             735 non-null    object
 8   Projectconsortium      739 non-null    object
 9   Projecttitel           739 non-null    object
 10  Publieke samenvatting  739 non-null    object
 11  Website link           739 non-null    object
 12  start_year             739 non-null    object
 13  start_date             739 non-null    object
dtypes: object(14)
memory usage: 86.6+ KB
None


### FUNDING AMOUNT <a class="anchor" id="funding"></a>

In [7]:
# Function to convert amount of money from string to float
def clean_subsidiebedrag(bedrag):
    # Remove valuta characters and dots (used for thousands), replace commas with dots (for decimals)
    bedrag = bedrag.replace('€', '').replace('.', '').replace(',', '.')
    return float(bedrag)

# Apply function to the column 'Subsidiebedrag'
df_rvo['Subsidiebedrag'] = df_rvo['Subsidiebedrag'].apply(clean_subsidiebedrag)

# Check
display(df_rvo.head(5))

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link,start_year,start_date
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,1549579.0,Kingspan Insulation B.V.,Kingspan Insulation B.V.,12 electrical-driven drying ovens,"Aanleiding\nCurrently, the Kingspan's Koolther...",https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,489962.0,Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",2D Circulair Biobased Bouwsysteem,Aanleiding\nOm circulaire bouwmethodes goedkop...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01
2,Versnelde klimaatinvesteringen industrie (VEKI),5. Restwarmte industrie,Lokale distributie van warmte,VEKI-19-01-01-Klimaatinvest. Indus.,VEKI120016,01-07-2020 - 31-03-2021,253118.0,4Pet Recycling B.V.,4PET Recycling B.V.,4PET op stoom,Aanleiding\n4PET Recycling verwerkt kunststof ...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01
3,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI220005,08-10-2020 - 31-12-2022,3000000.0,Coöperatie Koninklijke Cosun UA,Coöperatie Koninklijke Cosun U.A.*,8 Trapsverdamping,Aanleiding\nCosun Beet Company (CBC) beschikt ...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01
4,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI221016,01-01-2022 - 31-01-2024,200173.0,Delicia B.V.,Delicia B.V.,Add-on warmtepomp op bestaande koelinstallatie,Aanleiding\nIn de toekomst wil Delicia geen aa...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01


## TEXT PREPROCESSING FOR BERT <a class="anchor" id="text_prep"></a>

In [8]:
# Function for preprocessing textual data (for BERT)
def preprocess_text(text):
    # Lowercase the text and remove non-alphabetic characters
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    # Use regex to keep only words
    text = ' '.join(re.findall(r'\b[a-z]+\b', text))
    return text

# Apply the preprocessing to both 'Projecttitel' and 'Publieke samenvatting'
df_docs_rvo = df_rvo.copy()
df_docs_rvo['Projecttitel'] = df_docs_rvo['Projecttitel'].apply(preprocess_text)
df_docs_rvo['Publieke samenvatting'] = df_docs_rvo['Publieke samenvatting'].apply(preprocess_text)

In [9]:
# Combine 'Projecttitel' and 'Publieke samenvatting' into a new column 'text'
df_docs_rvo['text'] = df_docs_rvo['Projecttitel'] + ' ' + df_docs_rvo['Publieke samenvatting']

# Check
display(df_docs_rvo.head(5))
print(df_docs_rvo.shape)

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link,start_year,start_date,text
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,1549579.0,Kingspan Insulation B.V.,Kingspan Insulation B.V.,electricaldriven drying ovens,aanleiding currently the kingspans kooltherm i...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,electricaldriven drying ovens aanleiding curre...
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,489962.0,Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",d circulair biobased bouwsysteem,aanleiding om circulaire bouwmethodes goedkope...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,d circulair biobased bouwsysteem aanleiding om...
2,Versnelde klimaatinvesteringen industrie (VEKI),5. Restwarmte industrie,Lokale distributie van warmte,VEKI-19-01-01-Klimaatinvest. Indus.,VEKI120016,01-07-2020 - 31-03-2021,253118.0,4Pet Recycling B.V.,4PET Recycling B.V.,pet op stoom,aanleiding pet recycling verwerkt kunststof ve...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,pet op stoom aanleiding pet recycling verwerkt...
3,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI220005,08-10-2020 - 31-12-2022,3000000.0,Coöperatie Koninklijke Cosun UA,Coöperatie Koninklijke Cosun U.A.*,trapsverdamping,aanleiding cosun beet company cbc beschikt ove...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,trapsverdamping aanleiding cosun beet company ...
4,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI221016,01-01-2022 - 31-01-2024,200173.0,Delicia B.V.,Delicia B.V.,addon warmtepomp op bestaande koelinstallatie,aanleiding in de toekomst wil delicia geen aar...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01,addon warmtepomp op bestaande koelinstallatie ...


(739, 15)


## TRANSLATION <a class="anchor" id="translation"></a>

In [10]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Apply the function to the 'text' column and store the detected language in the 'language' column
tqdm.pandas(desc="Detecting languages")
df_docs_rvo['language'] = df_docs_rvo['text'].progress_apply(detect_language)

# Check
display(df_docs_rvo.head(5))

Detecting languages:   0%|          | 0/739 [00:00<?, ?it/s]

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link,start_year,start_date,text,language
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,1549579.0,Kingspan Insulation B.V.,Kingspan Insulation B.V.,electricaldriven drying ovens,aanleiding currently the kingspans kooltherm i...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,electricaldriven drying ovens aanleiding curre...,en
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,489962.0,Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",d circulair biobased bouwsysteem,aanleiding om circulaire bouwmethodes goedkope...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,d circulair biobased bouwsysteem aanleiding om...,nl
2,Versnelde klimaatinvesteringen industrie (VEKI),5. Restwarmte industrie,Lokale distributie van warmte,VEKI-19-01-01-Klimaatinvest. Indus.,VEKI120016,01-07-2020 - 31-03-2021,253118.0,4Pet Recycling B.V.,4PET Recycling B.V.,pet op stoom,aanleiding pet recycling verwerkt kunststof ve...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,pet op stoom aanleiding pet recycling verwerkt...,nl
3,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI220005,08-10-2020 - 31-12-2022,3000000.0,Coöperatie Koninklijke Cosun UA,Coöperatie Koninklijke Cosun U.A.*,trapsverdamping,aanleiding cosun beet company cbc beschikt ove...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,trapsverdamping aanleiding cosun beet company ...,nl
4,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI221016,01-01-2022 - 31-01-2024,200173.0,Delicia B.V.,Delicia B.V.,addon warmtepomp op bestaande koelinstallatie,aanleiding in de toekomst wil delicia geen aar...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01,addon warmtepomp op bestaande koelinstallatie ...,nl


In [11]:
# Function for translating text from Dutch to English
def translate_text(text):
    try:
        return GoogleTranslator(source='nl', target='en').translate(text)
    except Exception as e:
        return text

# Function for translating the 'text' column from this dataset from Dutch to English
def translate_dataset(df):
    # Check that required columns are present
    if 'language' not in df.columns or 'text' not in df.columns:
        raise ValueError("DataFrame must contain 'language' and 'text' columns")
    
    # Filter out rows where language is Dutch
    dutch_df = df[df['language'] == 'nl']
    
    # Translate the Dutch text to English in batches
    dutch_df['text'] = [translate_text(text) for text in tqdm(dutch_df['text'], desc="Translating")]
    
    # Update the original dataframe with the translated text
    df.update(dutch_df)
    
    return df

In [12]:
# Completely translate the 'text' column in the RVO dataset to English
df_translate = df_docs_rvo.copy()
df_translated_rvo = translate_dataset(df_translate)

Translating:   0%|          | 0/580 [00:00<?, ?it/s]

/var/folders/fx/2jd97zj90xv26bw8yfq7mjgr0000gn/T/ipykernel_29626/3783869340.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dutch_df['text'] = [translate_text(text) for text in tqdm(dutch_df['text'], desc="Translating")]


In [13]:
# Check
df_translated_rvo.head(10)

,Subsidie categorie,TKI/Thema,Innovatiethema('s),Subsidieregeling,Projectnummer,Looptijd,Subsidiebedrag,Penvoerder,Projectconsortium,Projecttitel,Publieke samenvatting,Website link,start_year,start_date,text,language
0,Versnelde klimaatinvesteringen industrie (VEKI),Directe effecten aan de schoorsteen (scope 1),Energie-efficiency,VEKI-23-01-01-Klimaatinvest. Indus.,VEKI523010,01-07-2023 - 31-12-2025,1549579.0,Kingspan Insulation B.V.,Kingspan Insulation B.V.,electricaldriven drying ovens,aanleiding currently the kingspans kooltherm i...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,electricaldriven drying ovens aanleiding curre...,en
1,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-22-03360660,14-02-2023 - 13-02-2025,489962.0,Sustainer Homes B.V.,"Sustainer Homes B.V., Technische Universiteit ...",d circulair biobased bouwsysteem,aanleiding om circulaire bouwmethodes goedkope...,https://projecten.topsectorenergie.nl/projecte...,2023,2023-01-01,d circular biobased construction system reason...,nl
2,Versnelde klimaatinvesteringen industrie (VEKI),5. Restwarmte industrie,Lokale distributie van warmte,VEKI-19-01-01-Klimaatinvest. Indus.,VEKI120016,01-07-2020 - 31-03-2021,253118.0,4Pet Recycling B.V.,4PET Recycling B.V.,pet op stoom,aanleiding pet recycling verwerkt kunststof ve...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,pet on steam pet recycling processes plastic p...,nl
3,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI220005,08-10-2020 - 31-12-2022,3000000.0,Coöperatie Koninklijke Cosun UA,Coöperatie Koninklijke Cosun U.A.*,trapsverdamping,aanleiding cosun beet company cbc beschikt ove...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,step evaporation reason cosun beet company cbc...,nl
4,Versnelde klimaatinvesteringen industrie (VEKI),4. Procesverbetering/energiebesparing,Energie-efficiency,VEKI-20-01-01-Klimaatinvest. Indus.,VEKI221016,01-01-2022 - 31-01-2024,200173.0,Delicia B.V.,Delicia B.V.,addon warmtepomp op bestaande koelinstallatie,aanleiding in de toekomst wil delicia geen aar...,https://projecten.topsectorenergie.nl/projecte...,2022,2022-01-01,add-on heat pump to existing cooling installat...,nl
5,PPS Toeslag,Energie en Industrie,2 Systeemintegratie- Elektrificatie en flexibi...,TKITOE-Energie en Industrie,TKITOEAmazing,01-09-2020 - 31-08-2024,936653.0,ISPT,"Dow Benelux B.V., Eurest Services B.V., Shell ...",additive manufacturing for zeroemission innova...,aanleiding satisfying the everincreasing globa...,https://projecten.topsectorenergie.nl/projecte...,2020,2020-01-01,additive manufacturing for zeroemission innova...,en
6,EZ Energie Innovatie,Transitieagenda Circulaire Bouweconomie,Transitieagenda Circulaire Bouweconomie,Kennis en Innovatie Agenda - Circulaire Econom...,KIACE-23-03577178,01-01-2024 - 31-12-2025,427114.0,Zavhy B.V.,"IMd Raadgevende Ingenieurs, Remmits GWW",additive manufacturing van circulaire betonnen...,aanleiding d concrete printing dcp is een tech...,https://projecten.topsectorenergie.nl/projecte...,2024,2024-01-01,additive manufacturing of circular concrete br...,nl
7,EZ Energie Innovatie,TKI BBE,Biotechnologische conversietechnologie,TSE-18-18-01-BBEG Innovatie,TBBE118009,19-09-2018 - 30-04-2021,239949.0,DSD,"DSD, Ginkgo Bioworks Netherlands B.V., Stichti...",advanced betaprocess met direct processing,aanleiding de drijfveer van dit project is het...,https://projecten.topsectorenergie.nl/projecte...,2018,2018-01-01,advanced beta process with direct processing T...,nl
8,PPS Toeslag,Energie en Industrie,2 Systeemintegratie- Elektrificatie en flexibi...,TKITOE-Energie en Industrie,TKITOESI-50-07c(SI-20-07c),01-01-2020 - 31-12-2021,732600.0,ISPT,"Dow Benelux B.V., Eurest Services B.V., Gasuni...",advanced gw water electrolysis,gr

In [14]:
# Detecting languages again from the translated texts and store the detected language in the column 'language'
tqdm.pandas(desc="Detecting languages")
df_translated_rvo['language'] = df_translated_rvo['text'].progress_apply(detect_language)

Detecting languages:   0%|          | 0/739 [00:00<?, ?it/s]

In [15]:
# Checking if all rows are in English now
all_english = df_translated_rvo['language'].eq('en').all()
if all_english:
    print("All rows are in English.")
else:
    print("Not all rows are in English.")

Not all rows are in English.


In [16]:
# Checking if the non-translated rows are actually not in English
non_eng = df_translated_rvo[df_translated_rvo['language'] != 'en']

# Print the non-English texts
print("Untranslatable texts in dataset:",len(non_eng))
print(non_eng['text'])

Untranslatable texts in dataset: 35
159    cost reduction industrial heat pumps rendabele...
194    demonstratieproject hybridedroger demonstratie...
210    draaitafelpers voor directe productie van baks...
217    duurzaam concentreren van aardappelsap met mem...
226    early adopter project energiezuinige voc verwi...
278    energiebesparing in industriele productieproce...
287    energy efficient affintiydriven molecular sepa...
290    energy efficient membrane based acetone recove...
321    forward osmosis membranes and modules developm...
324    freeze concentration vriesconcentratie een zui...
329    gas treatment revasin inventariseren van groot...
348    grip op drogen sensordata bieden houvast voor ...
393    hergebruik van latente warmte van sproeidroger...
396    higee efficint scheiden met centrifugeren proc...
405    implementation of solvent recovery in edible o...
413    industrile warmte terugwinning nieuwe modules ...
463    low cost storage of heat low cost storage of 

In [17]:
# Computing average text-length for non-English texts
total_length = 0
for text in non_eng['text']:
    total_length += len(text)

print("Average text length for non-eng texts:", total_length/len(non_eng['text']))

Average text length for non-eng texts: 6516.685714285714


In [18]:
# Drop all non-English texts
df_english_rvo = df_translated_rvo[df_translated_rvo['language'] == 'en']

# Show size of the dataset now
print(df_english_rvo.shape)

(704, 16)


In [19]:
# Preprocess text again (for capital letters that might have reoccurred after translation)
df_english_rvo['text'] = df_english_rvo['text'].apply(preprocess_text)

/var/folders/fx/2jd97zj90xv26bw8yfq7mjgr0000gn/T/ipykernel_29626/498494238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english_rvo['text'] = df_english_rvo['text'].apply(preprocess_text)


## EXPORT DATA <a class="anchor" id="export"></a>

In [20]:
# Exporting clean dataset to pickle file
df_english_rvo.to_pickle(f"../data/rvo_df_eng.pkl")